In [ ]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *

# qiskit-ibmq-provider has been deprecated.
# Please see the Migration Guides in https://ibm.biz/provider_migration_guide for more detail.
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session, Options

# Loading your IBM Quantum account(s)
service = QiskitRuntimeService(channel="ibm_quantum")

# Invoke a primitive. For more details see https://docs.quantum-computing.ibm.com/run/primitives
# result = Sampler().run(circuits).result()

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from qiskit import QuantumCircuit, Aer, execute
from qiskit.circuit.exceptions import CircuitError
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

# Define the Quantum Circuit
def create_quantum_circuit(num_qubits, depth):
    circuit = QuantumCircuit(num_qubits)

    for _ in range(depth):
        for qubit in range(num_qubits):
            circuit.rx(2 * np.pi * np.random.rand(), qubit)
            circuit.rz(2 * np.pi * np.random.rand(), qubit)
            target_qubit = (qubit + np.random.randint(1, num_qubits)) % num_qubits
            circuit.cx(qubit, target_qubit)

    return circuit


# Load your dataset from a txt file
dataset_path = 'SchedulingWithoutConflicts_n16_m3.txt'

# Read each line from the text file and process data
data_lists = []

with open(dataset_path, 'r') as file:
    lines = file.readlines()

# Ensure each line has a valid syntax
for i, line in enumerate(lines):
    try:
        line = line.strip()[1:-1]  # Remove the leading '[' and trailing ']'
        data_list = [list(map(float, sublist.split(','))) for sublist in line.split('] [')]
        if len(data_list) >= 2:  # Check if data_list has the required length
            data_lists.append(data_list)
        else:
            print(f"Error in line {i + 1}: Insufficient data in line")
            print(f"Line content: {line}")
    except Exception as e:
        print(f"Error in line {i + 1}: {e}")
        print(f"Line content: {line}")

        
# Check lengths before creating DataFrame
transaction_lengths_lengths = [len(data_list[0]) for data_list in data_lists]
optimal_assignment_lengths = [len(data_list[1]) if len(data_list) >= 2 else 0 for data_list in data_lists]

# Print lengths for lines where lengths are inconsistent
for i, (t_length, o_length) in enumerate(zip(transaction_lengths_lengths, optimal_assignment_lengths)):
    if t_length != 16 or o_length != 16:
        print(f"Line {i + 1}: transaction_length: {t_length}, optimal_assignment: {o_length}")
        print(f"Line content: {lines[i].strip()}")


# Create a DataFrame from the lists
df = pd.DataFrame({
    'transaction_length': [max(data_list[0]) for data_list in data_lists],  
    'optimal_assignment': [data_list[1] if len(data_list) >= 2 else [] for data_list in data_lists]
})


# Amplitude Encoding. use transaction lengths as amplitudes.    
#def amplitude_encoding(circuit, normalized_lengths, qubits):
#    for i, amplitude in enumerate(normalized_lengths):
#        circuit.ry(2 * np.arcsin(np.sqrt(amplitude)), qubits[i])

# Amplitude Encoding. use transaction lengths as amplitudes. 
def amplitude_encoding(circuit, normalized_lengths, qubits):
    #print("Amplitude Encoding executed")
    for i, amplitude in enumerate(normalized_lengths):
        if amplitude == 1.0:
            circuit.rx(0.0, qubits[i])  # If amplitude is 1, set angle to 0
        else:
            # Ensure amplitude is within the valid range [-1, 1]
            amplitude = max(min(amplitude, 1.0), -1.0)
            
            # Use arcsin only if amplitude is not NaN
            if not np.isnan(amplitude):
                circuit.ry(2 * np.arcsin(np.sqrt(amplitude)), qubits[i])
            else:
                print(f"Warning: NaN amplitude encountered at qubit {i}. Setting angle to 0.")
                circuit.rx(0.0, qubits[i])
        
        
# Create the VQC Model
class QuantumModel(nn.Module):
    def __init__(self, num_qubits):
        super(QuantumModel, self).__init__()
        self.num_qubits = num_qubits
        self.theta = nn.Parameter(torch.rand(num_qubits, requires_grad=True))

    def forward(self, quantum_circuit, params):
        # Execute quantum circuit and obtain expectation value
        quantum_circuit.assign_parameters({self.theta: params})
        return quantum_circuit.expectation_value()

    
    
# Train the quantum model using PyTorch's optimization techniques
num_samples = len(data_lists)
num_qubits = 4
depth = 3

model = QuantumModel(num_qubits)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


# Inside the training loop
for epoch in range(100):
    total_loss = 0.0  # Initialize total loss for the epoch
    for sample_index in range(num_samples):
        #print(f"Processing sample {sample_index} in epoch {epoch}")

        # Create a new quantum circuit for each sample
        quantum_circuit = create_quantum_circuit(num_qubits, depth)

        # Encode the amplitudes based on the normalized lengths
        amplitude_encoding(quantum_circuit, [max(data_lists[sample_index][0])], list(range(num_qubits)))

        # Convert the quantum circuit to a statevector simulator
        simulator = Aer.get_backend('statevector_simulator')

        # Set the parameters in the quantum circuit based on optimization results
        max_length = max(data_lists[sample_index][0])
        normalized_lengths = [length / max_length for length in data_lists[sample_index][0]]
        params = torch.tensor(normalized_lengths, dtype=torch.float32).requires_grad_(True)

        try:
            # Update the parameters in the quantum circuit
            for i in range(num_qubits):
                quantum_circuit.data[2 * i + 1][0].params = [params[i].item()]

            result = execute(quantum_circuit, simulator).result()
            statevector = result.get_statevector()

            # Calculate the expectation value directly without using the QuantumModel forward
            expectation_value = np.vdot(statevector, statevector).real
            loss = -torch.tensor(expectation_value, dtype=torch.float32, requires_grad=True)
        except CircuitError as e:
            print(f"Error in sample {sample_index}: {e}")
            continue

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()  # Accumulate the loss for this sample
    
    if epoch % 10 == 0:
        average_loss = total_loss / num_samples
        print(f'Epoch {epoch}, Average Loss: {average_loss}')
      

        
# Visualize the final Quantum Circuit
quantum_circuit = create_quantum_circuit(num_qubits, depth)
print("Quantum Circuit created")
amplitude_encoding(quantum_circuit, [max(data_lists[0][0])], quantum_circuit.qubits)
quantum_circuit.draw(output='mpl')
plt.show()

# Visualize the final statevector
result = execute(quantum_circuit, simulator).result()
statevector = result.get_statevector()
plot_histogram(result.get_counts())
plt.show()


In [32]:
# Save the model after training
torch.save(model.state_dict(), 'trained_model.pth')

# Load the model for inference
loaded_model = QuantumModel(num_qubits)
loaded_model.load_state_dict(torch.load('trained_model.pth'))


<All keys matched successfully>

In [33]:
#Prepare Input for Inference:
def encode_for_inference(circuit, length, qubits):
    amplitude_encoding(circuit, [length], qubits)

In [34]:
#Use the loaded model to perform inference on the quantum circuit.
def estimate_optimal_assignment(model, quantum_circuit, transaction_length):
    # Encode the transaction length
    encode_for_inference(quantum_circuit, transaction_length, list(range(num_qubits)))

    # Execute the quantum circuit
    simulator = Aer.get_backend('statevector_simulator')
    result = execute(quantum_circuit, simulator).result()
    statevector = result.get_statevector()

    # Calculate the expectation value
    expectation_value = np.vdot(statevector, statevector).real

    # Return the estimated optimal assignment
    return -expectation_value


In [35]:
# Estimate optimal assignment for a given transaction length
transaction_length = 16  # (what should be the value here ? ) transaction length
quantum_circuit = create_quantum_circuit(num_qubits, depth)

# Perform inference
estimated_optimal_assignment = estimate_optimal_assignment(loaded_model, quantum_circuit, transaction_length)

print(f"Estimated Optimal Assignment: {estimated_optimal_assignment}")


Amplitude Encoding executed
Estimated Optimal Assignment: -0.9999999999999999


In [37]:
# Assuming your model is already trained and stored in the 'model' variable

# Given transaction lengths
#given_transaction_lengths = [132.858,125.741,138.806,125.827,134.003,139.252,134.006,129.873,137.17,135.238,149.43,136.5,141.289,139.537,132.815,131.274]  # Replace with actual lengths
given_transaction_lengths = [315.688,235.425,382.758,236.392,328.601,387.794,328.629,282.025,364.31,342.529,502.579,356.756,410.764,391.003,315.199,297.816]

# Create a new quantum circuit
quantum_circuit = create_quantum_circuit(num_qubits, depth)

# Encode transaction lengths using the trained model
amplitude_encoding(quantum_circuit, given_transaction_lengths, list(range(num_qubits)))

# Execute the quantum circuit
simulator = Aer.get_backend('statevector_simulator')
result = execute(quantum_circuit, simulator).result()
statevector = result.get_statevector()

# Extract information and estimate optimal assignment (modify as needed)
estimated_assignment = some_function_to_extract_optimal_assignment(statevector)

# Print or use the estimated assignment
print(f"Estimated Optimal Assignment: {estimated_assignment}")


Amplitude Encoding executed
Traceback (most recent call last):
  Cell In[37], line 11
    amplitude_encoding(quantum_circuit, given_transaction_lengths, list(range(num_qubits)))
  Cell In[29], line 83 in amplitude_encoding
    circuit.ry(2 * np.arcsin(np.sqrt(amplitude)), qubits[i])
IndexError: list index out of range

Use %tb to get the full traceback.


In [43]:
# Assuming your model is already trained and stored in the 'model' variable

# Given transaction lengths
given_transaction_lengths =[132.858,125.741,138.806,125.827,134.003,139.252,134.006,129.873,137.17,135.238,149.43,136.5,141.289,139.537,132.815,131.274] 

# Create a new quantum circuit for each sample
quantum_circuit = create_quantum_circuit(num_qubits, depth)
print(f"Number of qubits in quantum circuit: {len(quantum_circuit.qubits)}")

# Encode the amplitudes based on the normalized lengths
amplitude_encoding(quantum_circuit, [max(data_lists[sample_index][0])], quantum_circuit.qubits)

# Print the size of the qubits list before and after the encoding
print(f"Number of qubits before encoding: {len(quantum_circuit.qubits)}")
amplitude_encoding(quantum_circuit, [max(data_lists[sample_index][0])], quantum_circuit.qubits)
print(f"Number of qubits after encoding: {len(quantum_circuit.qubits)}")

# Convert the quantum circuit to a statevector simulator
simulator = Aer.get_backend('statevector_simulator')
result = execute(quantum_circuit, simulator).result()
statevector = result.get_statevector()

# Extract information and estimate optimal assignment (modify as needed)
# Assuming the statevector represents a probability distribution over all possible assignments
assignment_probabilities = np.abs(statevector) ** 2
estimated_assignment = np.argmax(assignment_probabilities)


# Print or use the estimated assignment
print(f"Estimated Optimal Assignment: {estimated_assignment}")


quantum_circuit.draw(output='mpl')
plt.show()

# Visualize the final statevector
#result = execute(quantum_circuit, simulator).result()
#statevector = result.get_statevector()
#plot_histogram(result.get_counts())
#plt.show()


Number of qubits in quantum circuit: 4
Amplitude Encoding executed
Number of qubits before encoding: 4
Amplitude Encoding executed
Number of qubits after encoding: 4
Estimated Optimal Assignment: 14


In [45]:
# Assuming your model is already trained and stored in the 'model' variable

def extract_optimal_assignment(statevector):
    # Assuming the statevector represents a quantum superposition of different assignments
    # You need to define a logic to extract the optimal assignment based on the probabilities

    # For simplicity, let's assume that the optimal assignment corresponds to the basis state
    # with the highest probability in the statevector

    probabilities = np.abs(statevector) ** 2  # Calculate probabilities
    optimal_assignment = np.argmax(probabilities)  # Find the index with the highest probability

    # Convert the index to a binary array representing the basis state
    num_qubits = int(np.log2(len(statevector)))
    binary_optimal_assignment = format(optimal_assignment, f'0{num_qubits}b')
    
    # Convert the binary representation to a list of integers
    optimal_assignment_list = [int(bit) for bit in binary_optimal_assignment]

    return optimal_assignment_list

# Given transaction lengths
given_transaction_lengths = [132.858, 125.741, 138.806, 125.827, 134.003, 139.252, 134.006, 129.873,
                             137.17, 135.238, 149.43, 136.5, 141.289, 139.537, 132.815, 131.274]

# Create a new quantum circuit for each sample
quantum_circuit = create_quantum_circuit(num_qubits, depth)
print(f"Number of qubits in quantum circuit: {len(quantum_circuit.qubits)}")

# Encode the amplitudes based on the normalized lengths
amplitude_encoding(quantum_circuit, [max(given_transaction_lengths)], quantum_circuit.qubits)

# Print the size of the qubits list before and after the encoding
print(f"Number of qubits before encoding: {len(quantum_circuit.qubits)}")
amplitude_encoding(quantum_circuit, [max(given_transaction_lengths)], quantum_circuit.qubits)
print(f"Number of qubits after encoding: {len(quantum_circuit.qubits)}")

# Convert the quantum circuit to a statevector simulator
simulator = Aer.get_backend('statevector_simulator')
result = execute(quantum_circuit, simulator).result()
statevector = result.get_statevector()

# Extract the optimal assignment using the trained model
estimated_assignment = extract_optimal_assignment(statevector)

# Print or use the estimated assignment
print(f"Estimated Optimal Assignment: {estimated_assignment}")


Number of qubits in quantum circuit: 4
Amplitude Encoding executed
Number of qubits before encoding: 4
Amplitude Encoding executed
Number of qubits after encoding: 4
Estimated Optimal Assignment: [1, 1, 1, 1]


/tmp/ipykernel_131/1659587820.py:14: DeprecationWarning: The return type of saved statevectors has been changed from a `numpy.ndarray` to a `qiskit.quantum_info.Statevector` as of qiskit-aer 0.10. Accessing numpy array attributes is deprecated and will result in an error in a future release. To continue using saved result objects as arrays you can explicitly cast them using  `np.asarray(object)`.
  num_qubits = int(np.log2(len(statevector)))
